# Dados e bibliotecas

obs: caminho das pastas seguindo ambiente kaggle do desafio escolhido

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from keras.applications import EfficientNetB0
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
train = pd.read_csv('/kaggle/input/shopee-product-matching/train.csv')

In [14]:
# grupos do mesmo produto para análise de desempenho no treinamento
tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
train['target'] = train.label_group.map(tmp)

# Funções

In [15]:
def quant_batch(df, batch):
    ct = len(df) // batch
    ct += int(( (len(df)) % batch)!=0)
    return ct

In [16]:
def indices_por_batch(df, batch, index):
    indices = df[index*batch:(index+1)*batch]
    return indices

In [28]:
def obter_img_resize(df, batch, index, caminho = "/kaggle/input/shopee-product-matching/train_images/"):
    df_parcial = indices_por_batch(df, batch, index) 
    
    tamanho = len(df_parcial)
    matriz = np.zeros((tamanho, 256, 256, 3),dtype='float32')

    for i,(index,row) in enumerate(df_parcial.iterrows()):
        img = cv2.imread(caminho+row.image)
        matriz[i,] = cv2.resize(img,(256, 256)) 

    #matriz = matriz / 255 #normalizar imagem
    return matriz

In [18]:
import string
import re #replace
 
def removePunctuation(text):
    punc_translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    return text.translate(punc_translator)

def removeMedidas(text):
    return re.sub(r'kg|cm|gr|ml|xl', "", text)


def removeNumer(text):
    return re.sub(r"^[\d\s]+|[\d][.\d]+|[\d]", "", text)

def removeSpecialCaracter(text):
    return re.sub(r"^[@.,\\\/\+\-\|\[\]]!+()", "", text)

def removeSpace(text):
    return " ".join(text.split())

In [19]:
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.target,row[col]) )
        return 2*n / (len(row.target)+len(row[col]))
    return f1score

In [20]:
def formato_submissao(row):
    x = np.concatenate([row['predicao_efc'], row['predicao_tfidf'], row['predicao_hash']])
    return ' '.join( np.unique(x))

# Classificação através das imagens

Pesos do modelo baixado do link: <https://storage.googleapis.com/keras-applications/efficientnetb0_notop.h5>

Arquivo importado no ambiente kaggle como modelo keras

In [21]:
model = EfficientNetB0(include_top=False,
    weights='/kaggle/input/efcb0notop/keras/default/1/efficientnetb0_notop.h5',
    input_shape=(256,256,3),
    pooling='avg')

In [29]:
BATCH = 1500
quantidade = quant_batch(train, BATCH)
embeds = []
# obter embedding de forma parcial por questões de memória
for i in range(quantidade):
    imagens_array = obter_img_resize(train, BATCH, i)
    image_embeddings = model.predict(imagens_array)
    embeds.append(image_embeddings)

32/32 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step


In [ ]:
image_embeddings = np.concatenate(embeds) # colocar tudo em um array, ao realizar processo por batch

## Classificação por distância (KNN) entre embeddings

In [31]:
neighModel = NearestNeighbors(n_neighbors=50)
neighModel.fit(image_embeddings)

NearestNeighbors(n_neighbors=50)

In [41]:
predicao = []
CHUNK = 1500
THRESHOLD = 4.5

CTS = len(image_embeddings)//CHUNK
if len(image_embeddings)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(image_embeddings))
    distances, indices = neighModel.kneighbors(image_embeddings[a:b,])
    
    for k in range(b-a):
        IDX = np.where(distances[k,]< THRESHOLD)[0]
        IDS = indices[k,IDX]
        filtrar_vizinhos = train.iloc[IDS].posting_id.values
        predicao.append(filtrar_vizinhos)

array([[ 1.96529770e+00, -1.31772995e-01, -1.66103140e-01, ...,
         3.77804011e-01,  1.52934805e-01,  4.25890237e-01],
       [-1.58917367e-01, -1.61368757e-01, -3.90131287e-02, ...,
        -7.50639886e-02, -1.05371304e-01,  1.02647436e+00],
       [ 1.27433017e-02, -1.14663169e-01, -1.53293550e-01, ...,
        -2.64564529e-04, -5.29838279e-02,  7.12360859e-01]], dtype=float32)

In [36]:
train['predicao_efc'] = predicao
train.head(5)

,posting_id,image,image_phash,title,label_group,target,predicao_efc
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,[train_129225211],[train_129225211]
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,[train_3386243561],[train_3386243561]
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,[train_2288590299],[train_2288590299]
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,[train_2406599165],[train_2406599165]
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,[train_3369186413],[train_3369186413]


# TFIDF

Predição feita através da distância entre vetores das palavras

In [ ]:
# Limpeza de texto

train['title_clean'] = train['title'].str.lower()
train['title_clean'] = train['title_clean'].apply(removePunctuation)
train['title_clean'] = train['title_clean'].map(removeMedidas)
train['title_clean'] = train['title_clean'].apply(removeNumer)
train['title_clean'] = train['title_clean'].apply(removeSpecialCaracter)
train['title_clean'] = train['title_clean'].apply(removeNumer)
train['title_clean'] = train['title_clean'].apply(removeSpace)

In [ ]:
tfidf_vec = TfidfVectorizer(stop_words='english', 
                            binary=True, 
                            max_features=50000)
text_embeddings = tfidf_vec.fit_transform(train.title_clean).toarray()

Comparação de distância por lote por causa da memória 

In [ ]:
predicao_idf = []
CHUNK = 3500


CTS = len(train)//CHUNK
if len(train)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(train))
    
    # COSINE SIMILARITY DISTANCE
    cts = np.dot( text_embeddings, text_embeddings[a:b].T).T
    
    for k in range(b-a):
        IDX = np.where(cts[k,]>0.7)[0]
        filtrar_vizinhos = train.iloc[IDX].posting_id.values
        predicao_idf.append(filtrar_vizinhos)

In [ ]:
# adição ao df
train['predicao_tfidf'] = predicao_idf
train.head(5)

# Hash

Existem imagens com o mesmo hash na coluna image_hash

In [ ]:
tmp = train.groupby('image_phash').posting_id.agg('unique').to_dict()
train['predicao_hash'] = train.image_phash.map(tmp)
train.head()

# Avaliação do treinamento

In [ ]:
print("tfidf", train.apply(getMetric('predicao_tfidf'),axis=1).mean())

In [ ]:
print("hash", train.apply(getMetric('predicao_hash'),axis=1).mean())

In [37]:
print("efc", train.apply(getMetric('predicao_efc'),axis=1).mean())

efc 0.9713714285714287


# Aplicar classificação no dado de teste

In [39]:
test = pd.read_csv('/kaggle/input/shopee-product-matching/test.csv')

## Classificação através das imagens

Como a comparação é feita através das distâncias, os dados de treinamento não impactam, são utilizados apenas como referência da eficácia da técnica, pois o foco não é encontrar o label_group ou adaptar o modelo para identificar os rótulos. O foco é encontrar semelhança na distância entre os embeddings gerados pela imagem.

In [58]:
quantidade = quant_batch(test, BATCH)
embeds = []

for i in range(quantidade):
    imagens_array = obter_img_resize(test, BATCH, i, "/kaggle/input/shopee-product-matching/test_images/")
    image_embeddings = model.predict(imagens_array)
    embeds.append(image_embeddings)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step


In [59]:
if(quantidade>1):
    image_embeddings = np.concatenate(embeds) # colocar tudo em um array, ao realizar processo por batch

In [60]:
if(len(test)<50):
    n = len(test)
else:
    n = 50

neighModel = NearestNeighbors(n_neighbors=n)
neighModel.fit(image_embeddings)  #modelo feito pelo próprio dado de teste

NearestNeighbors(n_neighbors=3)

In [61]:
predicao = []
CHUNK = 1500

CTS = len(image_embeddings)//CHUNK
if len(image_embeddings)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(image_embeddings))
    distances, indices = neighModel.kneighbors(image_embeddings[a:b,])
    
    for k in range(b-a):
        IDX = np.where(distances[k,]< THRESHOLD)[0]
        IDS = indices[k,IDX]
        filtrar_vizinhos = test.iloc[IDS].posting_id.values
        predicao.append(filtrar_vizinhos)

In [62]:
test['predicao_efc'] = predicao
test.head(5)

,posting_id,image,image_phash,title,predicao_efc
0,test_2255846744,0006c8e5462ae52167402bac1c2e916e.jpg,ecc292392dc7687a,Edufuntoys - CHARACTER PHONE ada lampu dan mus...,[test_2255846744]
1,test_3588702337,0007585c4d0f932859339129f709bfdc.jpg,e9968f60d2699e2c,(Beli 1 Free Spatula) Masker Komedo | Blackhea...,[test_3588702337]
2,test_4015706929,0008377d3662e83ef44e1881af38b879.jpg,ba81c17e3581cabe,READY Lemonilo Mie instant sehat kuah dan goreng,[test_4015706929]


## TFIDF

In [ ]:
#pré-processamento para tfidf
test['title_clean'] = test['title'].str.lower()
test['title_clean'] = test['title_clean'].apply(removePunctuation)
test['title_clean'] = test['title_clean'].map(removeMedidas)
test['title_clean'] = test['title_clean'].apply(removeNumer)
test['title_clean'] = test['title_clean'].apply(removeSpecialCaracter)
test['title_clean'] = test['title_clean'].apply(removeNumer)
test['title_clean'] = test['title_clean'].apply(removeSpace)

In [ ]:
# tfidf
text_embeddings = tfidf_vec.transform(test.title_clean).toarray()

In [ ]:
predicao_idf = []
CHUNK = 3500


CTS = len(test)//CHUNK
if len(test)%CHUNK!=0: CTS += 1
for j in range( CTS ):
    
    a = j*CHUNK
    b = (j+1)*CHUNK
    b = min(b,len(test))
    
    # COSINE SIMILARITY DISTANCE
    cts = np.dot( text_embeddings, text_embeddings[a:b].T).T
    
    for k in range(b-a):
        IDX = np.where(cts[k,]>0.7)[0]
        filtrar_vizinhos = test.iloc[IDX].posting_id.values
        predicao_idf.append(filtrar_vizinhos)

In [ ]:
# adição ao df
test['predicao_tfidf'] = predicao_idf
test.head(5)

## Hash

In [ ]:
# classificação por hash
tmp = test.groupby('image_phash').posting_id.agg('unique').to_dict()
test['predicao_hash'] = test.image_phash.map(tmp)
test.head()

# Formatação para arquivo de envio

Nesse tipo de competição, necessário importar o arquivo de teste e gerar o arquivo de submissão com as previsões no próprio código. Durante a submissão, o arquivo de teste é substituído com o verdadeiro conjunto de teste.

In [ ]:
test['matches'] = test.apply(formato_submissao, axis = 1)

In [ ]:
sample = pd.read_csv('/kaggle/input/shopee-product-matching/sample_submission.csv') 

In [ ]:
sample = test[['posting_id','matches']]

In [ ]:
sample.to_csv(f'/kaggle/working/submission.csv',mode='a',index=False,header=True)

sub = pd.read_csv('submission.csv')
sub.head(6)